# Importing necessary files

In [ ]:

from utils import *
from annealing_schedules import *
from mcmc import *
from experiments import *

import pickle as cPickle

# Glauber (constant beta: 2.0) vs Metropolis (constant beta: 4.0)

In [ ]:
# Min. energy error for a few different alphas.
n = [100]
alphas = np.linspace(0.05, 2, 32)
N = 100000
n_exp = 10

beta_0 = 4.0
schedule = ConstantSchedule(beta_0)
b_v = schedule.get_schedule(N)

exps_M = MultiExperiments(n, alphas, N, b_v, n_exp, MCMC.metropolis )

beta_0 = 2.0
schedule_G = ConstantSchedule(beta_0)
b_v_G = schedule_G.get_schedule(N)

exps_G = MultiExperiments(n, alphas, N, b_v_G, n_exp, MCMC.glauber )



## Plot mean and std errors for different values of alpha

In [ ]:
sns.set(font_scale=1.7)
sns.set_palette("deep")

fsave = 'Plots/Alpha_MCMC_Glauber_Const_Beta_4_2_longer'
plot_error([exps_M.get_stats("min_energy_errors"), 
            exps_G.get_stats("min_energy_errors")], x_axis_vector=alphas,
           xlabel="alpha", legend=["Min energy error (Metropolis)", 
                                   "Min energy error (Glauber)"], fsave=fsave)

fpickle = 'alpha_beta_4M_2G_longer'
dump_alpha_comparison( exps_G, exps_M, fpickle)

## Compare final sample error against min energy sample error

In [ ]:
fsave = 'Plots/Glauber_Beta_2_Min_Energy_Final_Error_longer'
plot_error([exps_G.get_stats("min_energy_errors"),
            exps_G.get_stats("final_errors"),], x_axis_vector=alphas,
           xlabel="alpha", legend=["Min energy error (Glauber)",
                                   "Final error (Glauber)"], fsave=fsave)

fsave = 'Plots/Metropolis_Beta_4_Min_Energy_Final_Error_longer'
plot_error([exps_M.get_stats("min_energy_errors"),
            exps_M.get_stats("final_errors"),], x_axis_vector=alphas,
           xlabel="alpha", legend=["Min energy error (Metropolis)",
                                   "Final error (Metropolis)"], fsave=fsave)



# Glauber (constant beta: 2.0) vs Metropolis (logarithmic beta), N=100000 steps

In [ ]:
# Min. energy error for a few different alphas.
n = [100]
alphas = np.linspace(0.05, 2, 32)
N = 100000
n_exp = 10

t_start = 2.0
t_freeze = 0.15
log_mul_schedule = LogarithmicMultiplicativeSchedule( T_start=t_start , T_freeze=t_freeze , alpha=6.00)
schedule = log_mul_schedule
b_v = schedule.get_schedule(N)

exps_M1 = MultiExperiments(n, alphas, N, b_v,  n_exp, MCMC.metropolis )

beta_0 = 2.0
schedule_G = ConstantSchedule(beta_0)
b_v_G = schedule_G.get_schedule(N)

exps_G1 = MultiExperiments(n, alphas, N, b_v_G,  n_exp, MCMC.glauber )



## Plot mean and std errors for different values of alpha

In [ ]:
fsave = 'Plots/Alpha_MCMC_Glauber_Log_Const_Beta_2'
plot_error([exps_M1.get_stats("min_energy_errors"), 
            exps_G1.get_stats("min_energy_errors")], x_axis_vector=alphas,
           xlabel="alpha", legend=["Min energy error (Metropolis)", 
                                   "Min energy error (Glauber)"], fsave=fsave)

fpickle = 'alpha_beta_logM_2G'
dump_alpha_comparison( exps_G1, exps_M1, fpickle)

## Compare Glauber (constant beta: 2.0),  Metropolis (constant beta: 4.0),  Metropolis (logarithmic beta), N=100000 steps 

In [ ]:
fsave = 'Plots/Alpha_MCMC_4_Glauber_2_MCMC_Log'
plot_error([exps_M.get_stats("min_energy_errors"), 
            exps_G1.get_stats("min_energy_errors"),
           exps_M1.get_stats("min_energy_errors")], x_axis_vector=alphas,
           xlabel="alpha", legend=["Min energy error (Metropolis-Beta-4)", 
                                   "Min energy error (Glauber-Beta-2)",
                                   "Min energy error (Metropolis-Log)"], fsave=fsave)

fpickle = 'alpha_MCMC_4_Glauber_2_MCMC_Log'

save_object = defaultdict(defaultdict)
save_object['exp_G'] = exps_G1
save_object['exp_M1'] = exps_M
save_object['exp_M2'] = exps_M1

    
file_handler = open("%s.pkl" %(fpickle),"wb")
cPickle.dump(save_object,file_handler)
file_handler.close()


# Plotting error trend over time

In [ ]:
def plot_error_trend(e_vectors, title="", legend=[],
               xlabel="Time", fsave=' '):
    """Plots error over time (or the values in x_axis_vector). Inputs are
    lists of vectors to allow comparisons."""
    
    if not legend:
        legend = [" "] * len(e_vectors)
    plt.figure(figsize=(24,8))
    plt.hlines([0.1, 0.2, 0.3, 0.4, 0.5],0, len(e_vectors[0]),
               linestyles="--", color="lightgray")
    for i, e_vector in enumerate(e_vectors):
        plt.plot( np.arange(len(e_vector)), e_vector, label=legend[i])
    plt.legend(loc='upper right')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Error")
    plt.ylim(-0.05, 0.75)
    if len(fsave) > 0:
        plt.savefig(fsave + '.pdf')
        
def get_error_vectors_legend( exp, indx_low, indx_high ):
    legends = []
    e_vectors = []
    for indx in range( indx_low, indx_high ):
        alpha = exp.parameters[indx][1]
        legends.append( 'Alpha %.3f' %(alpha))
        e_vectors.append( exp.experiments[indx][0].e_v )
    return e_vectors, legends


fpickle = 'alpha_MCMC_4_Glauber_2_MCMC_Log'
file = open("%s.pkl" %(fpickle),"rb")
save_object = cPickle.load(file)
exps_G = save_object['exp_G']
exps_M = save_object['exp_M1']
exps_M1 = save_object['exp_M2']
file.close()

e_vectors, legends = get_error_vectors_legend( exps_M1, indx_low=12, indx_high=18 )
title='Error trend for different values of alpha for Metropolis with logarithmic schedule'
fsave='Plots/Metropolis_log_error_trend'
plot_error_trend(e_vectors, title=title, legend=legends, fsave=fsave)


e_vectors, legends = get_error_vectors_legend( exps_M, indx_low=12, indx_high=18 )
title='Error trend for different values of alpha for Metropolis with constant cooling schedule'
fsave='Plots/Metropolis_beta_4_error_trend'
plot_error_trend(e_vectors, title=title, legend=legends, fsave=fsave)

e_vectors, legends = get_error_vectors_legend( exps_G, indx_low=12, indx_high=18 )
title='Error trend for different values of alpha for Glauber with constant cooling schedule'
fsave='Plots/Glauber_beta_2_error_trend'
plot_error_trend(e_vectors, title=title, legend=legends, fsave=fsave)

## Plotting error trend for Metropolis (constant beta, logartihmic) and Glauber on same graph

In [ ]:

e_vectors_m1, legends_m1 = get_error_vectors_legend( exps_M1, indx_low=12, indx_high=18 )
e_vectors_m, legends_m = get_error_vectors_legend( exps_M, indx_low=12, indx_high=18 )
e_vectors_g, legends_g = get_error_vectors_legend( exps_G, indx_low=12, indx_high=18 )

e_vectors = [e_vectors_m1[0], e_vectors_m[0], e_vectors_g[0], e_vectors_m1[-1], e_vectors_m[-1], e_vectors_g[-1]]
legends = [ 'Metropolis-log ' + legends_m1[0],
            'Metropolis-beta-4 ' + legends_m[0],
            'Glauber-beta-2 ' + legends_m[0],
            'Metropolis-log ' + legends_m1[-1],
            'Metropolis-beta-4 ' + legends_m[-1],
            'Glauber-beta-2 ' + legends_m[-1],
          ]
title='Error trend for different values of alpha for Glauber with constant cooling schedule'
fsave='Plots/Error_trend_MCMC_log_MCMC_4_Glauber_2'
plot_error_trend(e_vectors, title=title, legend=legends, fsave=fsave)